!pip install psutil GPUtil pyudev

In [28]:
import psutil

import subprocess

def get_cpu_info():
    import sys,subprocess
    if sys.platform == 'linux':
        command = 'lscpu | grep "Model name"'
    elif sys.platform == 'win32':
        command = "wmic cpu get name"
    else:
        return None
    cpu_info = subprocess.run(command, shell=True, capture_output=True, text=True)
    cpu_info = str(cpu_info.stdout)
    cpu_info = cpu_info.split(':', 1)[1].strip()
    return cpu_info

def get_gpu_info():
    import GPUtil
    gpus = GPUtil.getGPUs()
    if gpus:
        for gpu in gpus:
            return (gpu.name)
    else:
        return None

def get_vector_card_info():
    try:
        # Run lspci and get the output
        lspci_output = subprocess.check_output(['lspci']).decode('utf-8')
        vector_cards = []
        for line in lspci_output.split('\n'):
            if 'Vector' in line:  # Adjust this string to match your vector card's identifier
                vector_cards.append(line)
        return str(vector_cards[0]) if vector_cards else None
    except Exception as e:
        return f"Error retrieving vector card info: {e}"

def print_hardware_info():
    cpu_info = get_cpu_info()
    gpu_info = get_gpu_info()
    vector_card_info = get_vector_card_info()
    
    if cpu_info:
        cpu_info = cpu_info.split(':', 1)[1].strip()
        print(f"CPU: {cpu_info}")
    if gpu_info:
        print(f"GPUs: {gpu_info[0]}")
    if vector_card_info:
        vector_card_info = ' '.join(vector_card_info.split()[1:])
        print(f"Vector Cards: {vector_card_info}")
    if not any([cpu_info, gpu_info, vector_card_info]):
        print("No hardware information found.")

# Integrate with Qiskit
from qiskit import BasicAer, execute, QuantumCircuit

# Your quantum circuit code
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# Execute on BasicAer's dm_simulator
simulator = BasicAer.get_backend('dm_simulator')
job = execute(qc, simulator)
result = job.result()
# Print the result of the execution
print(result.results[0])

# Print hardware information
print_hardware_info()


ExperimentResult(shots=0, success=True, meas_level=2, data=ExperimentResultData(partial_probability={'00': 0.5, '01': 0.0, '10': 0.0, '11': 0.5}, coeffmatrix=[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.25], densitymatrix=[[0.5+0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0.5+0.j]]), header=QobjExperimentHeader(clbit_labels=[['meas', 0], ['meas', 1]], creg_sizes=[['meas', 2]], global_phase=0.0, memory_slots=2, metadata={}, n_qubits=2, name='circuit-129', qreg_sizes=[['q', 2]], qubit_labels=[['q', 0], ['q', 1]]), status=DONE, name='circuit-129', number_of_clock_cycles=3, processing_time_taken=0.00020837783813476562, running_time_taken=0.0010726451873779297)
CPU: 12th Gen Intel(R) Core(TM) i9-12900
GPUs: NVIDIA T400 4GB


In [19]:
vector_card_info = '52:00.0 Co-processor: NEC Corporation Vector Engine 1.0 (rev 01)'
print(vector_card_info)

52:00.0 Co-processor: NEC Corporation Vector Engine 1.0 (rev 01)


In [20]:
# Split the string by spaces and rejoin it omitting the first part


print(vector_card_info)

Co-processor: NEC Corporation Vector Engine 1.0 (rev 01)


In [11]:
import sys

In [14]:
sys.platform

'linux'

In [15]:
'win32'

'win32'

In [29]:
import platform
cpu_model = platform.processor()
print("CPU model:", cpu_model)

CPU model: x86_64


In [32]:
!pip install py-cpuinfo


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [33]:
import cpuinfo
processor_name = cpuinfo.get_cpu_info()['brand_raw']
print(f"Processor name: {processor_name}")

Processor name: 12th Gen Intel(R) Core(TM) i9-12900


In [30]:
def get_gpu_info():
    import GPUtil
    GPU = None
    gpus = GPUtil.getGPUs()
    if gpus:
        for gpu in gpus:
            GPU = gpu.name
        return GPU
    else:
        return None

GPU = get_gpu_info()

In [31]:
GPU

'NVIDIA T400 4GB'

In [36]:
def get_vector_card_info():
    import subprocess
    try:
        # Run lspci and get the output
        lspci_output = subprocess.check_output(['lspci']).decode('utf-8')
        vector_cards = []
        for line in lspci_output.split('\n'):
            if 'Vector' in line:  # Adjust this string to match your vector card's identifier
                vector_cards.append(line)
        vector_card_info = str(vector_cards[0])
        vector_card_info = ' '.join(vector_card_info.split()[1:])
        return vector_card_info if vector_cards else None
    except Exception as e:
        return f"Error retrieving vector card info: {e}"

vector_card_info = get_vector_card_info()

In [37]:
vector_card_info

'Error retrieving vector card info: list index out of range'

In [11]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler
import logging
import requests

requests.adapters.DEFAULT_RETRIES = 5  # Increase the number of retries
requests.adapters.DEFAULT_TIMEOUT = 60  # Increase the timeout

logging.basicConfig(level=logging.DEBUG)

# Initialize the Qiskit Runtime Service using the API key
service = QiskitRuntimeService(channel="ibm_quantum", token="127e23318bfacf5fb872cca46ce3937d09f844ad1cb596f97a3eea4ea9e445b1c6905eac9af2cf7d01bb10868c7aac70da1cefd4db724b0a53f4dc57b5d8d95c")

# Get a backend
backend = service.least_busy(min_num_qubits=2, simulator=False)
print("Running on backend:", backend)

# Create a quantum circuit with 2 qubits and 2 classical bits
qc = QuantumCircuit(2, 2)

# Apply a Hadamard gate on qubit 0
qc.h(0)

# Apply a CNOT gate (qubit 0 -> qubit 1)
qc.cx(0, 1)

# Measure both qubits
qc.measure([0, 1], [0, 1])

# Transpile the circuit for the selected backend
transpiled_qc = transpile(qc, backend)

# Execute the circuit using Qiskit Runtime Service
with Session(service=service, backend=backend) as session:
    sampler = Sampler(session=session)
    job = sampler.run(circuits=transpiled_qc)

    # Get the result and print the counts
    result = job.result()
    counts = result.quasi_dists[0].binary_probabilities()
    print("Result:", counts)


DEBUG:urllib3.util.retry:Converted retries value: 0 -> Retry(total=0, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.util.retry:Converted retries value: 0 -> Retry(total=0, connect=None, read=None, redirect=None, status=None)
session._log_request_info:DEBUG:2024-08-09 17:16:59,414: Endpoint: https://auth.quantum-computing.ibm.com/api/version. Method: GET. 
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): auth.quantum-computing.ibm.com:443
session.increment:DEBUG:2024-08-09 17:17:04,836: Retrying method=GET, url=/api/version, status=None, error=HTTPSConnectionPool(host='auth.quantum-computing.ibm.com', port=443): Read timed out. (read timeout=5.0), data=None, headers=None
DEBUG:urllib3.util.retry:Incremented Retry for (url='/api/version'): PostForcelistRetry(total=4, connect=3, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (2): auth.quantum-computing.ibm.com:443
DEBUG:urllib3.connectionpool:https://au

Running on backend: <IBMBackend('ibm_kyoto')>


DEBUG:urllib3.connectionpool:https://api.quantum.ibm.com:443 "GET /runtime/backends/ibm_kyoto/properties HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.quantum.ibm.com:443 "GET /runtime/backends/ibm_kyoto/defaults HTTP/1.1" 200 None
DEBUG:stevedore.extension:found extension EntryPoint(name='default', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:DefaultInitPassManager', group='qiskit.transpiler.init')
DEBUG:stevedore.extension:found extension EntryPoint(name='default', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:DefaultLayoutPassManager', group='qiskit.transpiler.layout')
DEBUG:stevedore.extension:found extension EntryPoint(name='dense', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:DenseLayoutPassManager', group='qiskit.transpiler.layout')
DEBUG:stevedore.extension:found extension EntryPoint(name='noise_adaptive', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:NoiseAdaptiveLayoutPassManager', group='qiskit.trans

Result: {'10': -0.4625005171760853, '11': 0.9406405222304194, '00': 0.9837109530138848, '01': -0.46185095806821885}


DEBUG:urllib3.connectionpool:https://api.quantum.ibm.com:443 "PATCH /runtime/sessions/ctv03fx66x8g008q3bag HTTP/1.1" 204 0


In [12]:
result

SamplerResult(quasi_dists=[{2: -0.462500517176085, 3: 0.940640522230419, 0: 0.983710953013885, 1: -0.461850958068219}], metadata=[{'shots': 4000, 'circuit_metadata': {}, 'readout_mitigation_overhead': 17.777004844631072, 'readout_mitigation_time': 0.055606876499950886, 'warning': 'Optimization level clipped from 3 to 1\nOptimization level clipped from 3 to 1'}])